Projet Optimisation Groupe 2 : STOCK 
DESJONQUERES Henri, MARZIN Ewen 

I. Etude du problème d'optimisation

1. 

La partie $  c^{T} \cdot r =  \displaystyle \sum_{j=1}^{n} c_j * r_j$ correspond aux dépenses totales d'achat des matières premières. 

$\min \{ q, d \}$ correspond au nombre de produit vendu qui est en fait bien le minimum entre le nombre de produit fabriqués et la quantité prévisionnelle de vente : en effet si on a fabriqué trop peu de produits par rapport à la prévision de ventes, on ne pourra vendre qu'au plus tous les produits fabriqués et inversemment si on a fabriqué plus de produit que la quantité prévisionnelle des ventes alors on vend un nombre de produits qui correpond à cette prévision de ventes. 

La partie $ v^{T} \cdot \min (q, d) = \displaystyle \sum_{j=1}^{n} v_j * \min (q_j, d_j)$ correspond alors au total de recettes des ventes des ventes des produits. 
La différence de la seconde moins la première nous donne donc le gain final de la boulangerie (recettes - dépenses)

2. 

La fonction minimum est une fonction qui n'est pas différentiable donc cela complexifie la résolution du problème d'optimisation 

3. 

On considère la fonction $h : (d,q) \rightarrow h(d,q)$ définie par pour tout $i \in \llbracket ~1;p~ \rrbracket$ : $h_i (d,q) = \frac{q_i \cdot e^{-\alpha \cdot q_i} + d_i \cdot e^{-\alpha \cdot d_i}}{e^{-\alpha \cdot q_i} + e^{-\alpha \cdot d_i} }$ , avec $\alpha >> 1$. 

Comme $\alpha >> 1$, si $q_i > d_i$ alors on a $ e^{-\alpha \cdot d_i} >> e^{-\alpha \cdot q_i}$ (car $e^{\alpha \cdot (q_i - d_i)}  \rightarrow + \infty$ lorsque $\alpha \rightarrow +\infty$)

Ainsi on a dans ce cas : $h_i(d,q) = \frac{q_i \cdot e^{- \alpha \cdot (q_i - d_i)} + d_i}{e^{- \alpha \cdot (q_i - d_i)} + 1 }$, et avec $e^{- \alpha \cdot (q_i - d_i)} << 1$ on obtient que : 
$h_i(d,q) = \frac{d_i + o(1)}{1+o(1)} ≈ d_i = \min(d_i,q_i)$

Si $q_i < d_i$ un raisonnement similaire nous donne dans ce cas : 
$h_i(d,q) = \frac{d_i + o(1)}{1+o(1)} ≈ q_i = \min(d_i,q_i)$

Et si $d_i = q_i$, on obtient directement $h_i(d,q) = q_i = d_i = \min(q_i, d_i)$

On a donc bien une bonne approximation de la fonction min avec la fonction h. 
Il est plus intéressant de considérer ce problème là car la fonction h est $C^1$ comme composée de fonctions $C^1$ 

4. 

En posant $z = (q, r)$, les variables de décision, soit $z = (z_1, z_2), avec z_1 = q \in \mathbb (R^p)_+$ et $z_2 = r \in \mathbb (R^m)_+$, on a alors $n = p + m$ variables de décisions.

Les contraintes s'écrivent alors : 
$c(z) = (- z_1, - z_2, Az_1 - z_2) \leq 0$

Et on cherche à résoudre le problème suivant, avec $f(z) = c^T \cdot z_2 - v^T \cdot h(z_1,d)$ la fontion objectif à minimiser (pour maximiser le profit = -$f(z)$, on minimise f)

$$ \min_{c(z) \leq 0} f(z) $$

II. Etude et résolution numérique 

5. 

Comme on est dans le cadre d'un problème d'optimisation sous contraintes linéaires, on pourrait penser à la méthode du simplexe, mais celle-ci necessite une égalité dans les contraintes à la place d'une inégalité. 

On peut donc utiliser la méthode SLQS pour résoudre le problème numériquement

II. Etude et résolution numérique

6. 

In [ ]:
import numpy as np
import scipy 
alpha = 0.1
c = 10**-3 *np.array([30, 1, 1.3, 4, 1])
v = np.array([0.9, 1.5, 1.1])
d = np.array([400, 67, 33])
A = np.array([3.5, 2, 1], [250, 80, 25], [0, 8, 3], [0, 40, 10], [0, 8.5, 0])

def h(q, d, alpha) :
    hp = np.zeros(len(q))
    for i in range(len(q)): 
        hp[i] = (q[i]*np.exp(-alpha*q[i]) - d[i]*np.exp(-alpha*d[i]))/(np.exp(-alpha*q[i]) + np.exp(-alpha*d[i]))
    return hp

def f(q, r, d, c, v, alpha) : 
    hp = h(q, d, alpha)
    f = 0 
    for k in range(len(c)): 
        f += c[k]*r[k]                                      ### on ajoute c^T*r = np.sum(c[k]*r[k])###
    for j in range(len(q)): 
        f -= v[j]*hp[j]                                      ### on soustrait v^T*h = np.sum(v[j]*h[j])###
    return f





III. Etude du problème non régularisé 

8. (a) On a toujours $z = (z_1, z_2) = (q, r) \in \mathbb R^p x R^m$ et $n = p+m$ le nombre de ces variables de décisions. 

On a toujours les contraintes $c(z)$ telles que : 
$c(z) = (- z_1, - z_2, Az_1 - z_2) \leq 0$

Néanmoins, la fonction à minimiser s'écrit cette fois sous la forme :
$f(z) = c^T \cdot z_2 - v^T \cdot min\{ z_1, d \}$

(b) Supposons par l'absurde que la contrainte (1) : $Az_1 - z_2 = Aq - r \leq 0$, n'est pas active à l'optimum. On note $z^* = (q^*, r^*)$ le minimiseur. 

On a alors l'existence d'un indice $i_0 \in \llbracket 1 ~;~ m \rrbracket$ tel que $(Aq^*)_{i_0} > r^*_{i_0}$. 

On pose $\~{r} = (r^*_1, ..., r^*_{i_0 -1}, (Aq*)_{i_0}, r^*_{i_0 +1}, ..., r^*_m)$, on a alors $Aq^* \leq \~{r}~$ et comme $(Aq^*)_{i_0} > r^*_{i_0}$ : 

$v^T \cdot min\{q^*, d\} - c^T \cdot \~{r} > v^T \cdot min\{q^*, d\} - c^T \cdot r*$

Soit $Aq^* \leq \~{r}$, $q^* \geq 0$, $\~{r} \geq 0$ et $f(q^*, \~{r}) < f(q^*, r^*)$. 

CONTRADICTION avec l'hypothèse de $(q^*,r^*)$ minimiseur.

On conlut que la contrainte est donc bien active à l'optimum

(c)